In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from deepsvg.svglib.svg import SVG

from deepsvg import utils
from deepsvg.svg_dataset import SVGDataset, load_dataset

import os
import ntpath
import re
from tqdm import tqdm
import pickle
import random

from cairosvg import svg2png
from PIL import Image
import cv2

import pandas as pd
import numpy as np

In [3]:
pretrained_folder = "pretrained/serifs"
pretrained_file = "serifs_labelled_256.pth.tar"
pretrained_path = os.path.join(pretrained_folder, pretrained_file)

from configs.deepsvg.hierarchical_ordered_gest_labelled import Config

cfg = Config()
dataset = load_dataset(cfg)

In [4]:
# extract dataframe
df = dataset.df

# create variables to save image paths
png_paths = []
jpg_paths = []

# create folder to save images
dataset_folder = os.path.dirname(cfg.meta_filepath)
png_folder = os.path.join(dataset_folder, 'pngs')
os.makedirs(png_folder, exist_ok = True)

# create corresponding jpg folder
jpg_folder = os.path.join(dataset_folder, 'jpgs')
os.makedirs(jpg_folder, exist_ok = True)

In [5]:
# generate images

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    # get id
    id1 = row["id"]
    
    # save png
    path = os.path.join(cfg.data_dir, id1+'.svg')
    png_path = os.path.join(png_folder, id1+'.png')
    with open(path, 'r') as f:
        path_string =f.read()
    svg2png(bytestring=path_string, write_to = png_path, scale = 0.32)
    png_paths.append(png_path)

100%|██████████| 24366/24366 [00:31<00:00, 771.42it/s]


In [6]:
def pure_pil_alpha_to_color_v2(image, color=(255, 255, 255)):
    """Alpha composite an RGBA Image with a specified color.

    Simpler, faster version than the solutions above.

    Source: http://stackoverflow.com/a/9459208/284318

    Keyword Arguments:
    image -- PIL RGBA Image object
    color -- Tuple r, g, b (default 255, 255, 255)

    """
    image.load()  # needed for split()
    background = Image.new('RGB', image.size, color)
    background.paste(image, mask=image.split()[3])  # 3 is the alpha channel
    return background

In [7]:
# png to jpg
for png_path in tqdm(png_paths):
    # path to jpg image
    jpg_path = re.sub('png', 'jpg', png_path)
    jpg_paths.append(jpg_path)
    
    # convert png to jpg and save
    png = Image.open(png_path)
    jpg = pure_pil_alpha_to_color_v2(png, color=(255, 255, 255))
    jpg.save(jpg_path)

100%|██████████| 24366/24366 [00:06<00:00, 3576.57it/s]
